# Setup

In [1]:
# give colab permission to access drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# required for odeformer, although it's annoying to run these so i made the required changes in odeformer
%pip install torch==2.0.0
%pip install numpy==1.23.5

# changed arg weights_only = False in torch.load in sklearn_wrapper to fix a loading error from running dstr = odeformer.model.SymbolicTransformerRegressor, i.e.
# model = torch.load(model_path, weights_only=False)

# also changed np.infty to np.inf

In [2]:
# replace with your drive paths to MIVLE repo, odeformer repo, and shared probing_data folder
repo_path = '/content/drive/MyDrive/aisc/repos/MIVLDE/subteams/LLMProbing'
odeformer_path = '/content/drive/MyDrive/aisc/repos/odeformer'
samples_path = '/content/drive/MyDrive/aisc/probing_data/samples/derivatives'
activations_path = '/content/drive/MyDrive/aisc/probing_data/activations/derivatives'
probes_path = '/content/drive/MyDrive/aisc/probing_data/probes/derivatives'

In [3]:
import sys
import importlib
sys.path.append(repo_path)
sys.path.append(odeformer_path)
import numpy as np
import os
import pickle

In [13]:
import odeformer.model
dstr = odeformer.model.SymbolicTransformerRegressor(from_pretrained=True)
model_args = {'beam_size': 10, 'beam_temperature': 0.8} # higher beam temp seems to be better for avoiding nans in solutions
dstr.set_model_args(model_args)

Downloading...
From (original): https://drive.google.com/uc?id=1L_UZ0qgrBVkRuhg5j3BQoGxlvMk_Pm1W
From (redirected): https://drive.google.com/uc?id=1L_UZ0qgrBVkRuhg5j3BQoGxlvMk_Pm1W&confirm=t&uuid=51ffcd68-22cc-49e0-8554-307ce91b2046
To: /content/odeformer.pt
100%|██████████| 465M/465M [00:06<00:00, 73.3MB/s]


Loaded pretrained model


# Generate samples

In [4]:
from src.sample_generation import ManualSamplesGenerator
msg = ManualSamplesGenerator(samples_path=samples_path)

In [6]:
# Times array
t_values = np.linspace(1, 10, 50)

# Exponential parameters
c_values = np.linspace(1,10, 10)
a_values = np.linspace(0,10,51)


In [7]:
# Generate exponential samples
msg.generate_exponential_samples(t_values, c_values, a_values)

Generating exponential samples:   0%|          | 0/510 [00:00<?, ?it/s]

Saving generated samples: 510it [00:05, 97.46it/s] 


[INFO] Data generation complete. Saved 510 exponential samples to /content/drive/MyDrive/aisc/probing_data/samples/derivatives


In [9]:
# Inspect an exponential sample to see its keys and some data
samples_dir = os.fsencode(samples_path)
for sample_file in os.listdir(samples_dir):
  sample_name = os.fsdecode(sample_file)
  if "exp" in sample_name:
    exp_sample_path = os.path.join(samples_path, sample_name)
    with open(exp_sample_path, 'rb') as f:
        exp_sample = pickle.load(f)
    print(exp_sample.keys())
    print(exp_sample['expression'])
    print(exp_sample['feature_dict'])
    break

dict_keys(['times', 'trajectory', 'parameters', 'feature_dict', 'expression'])
1.0 * np.exp(-0.0 * t)
{'exponential': 1, 'hyperbolic': 0, 'derivative_0': -0.0, 'derivative_3': -0.0}


# Activation extraction

In [10]:
from src.activation_extraction import ActivationsExtractor

In [11]:
act_extractor = ActivationsExtractor()

In [14]:
act_extractor.extract_activations(dstr, samples_path, activations_path, layers_to_extract=['ffn'])

Extracting Activations: 100%|██████████| 510/510 [1:03:51<00:00,  7.51s/it]


[INFO] Activation extraction complete. Activations saved to /content/drive/MyDrive/aisc/probing_data/activations/derivatives


In [7]:
# Inspect a manual activation and display its keys and some values
manual_acts_dir = os.fsencode(activations_path)
for acts_file in os.listdir(manual_acts_dir):
  acts_name = os.fsdecode(acts_file)
  manual_acts_path = os.path.join(activations_path, acts_name)
  with open(manual_acts_path, 'rb') as f:
      manual_acts = pickle.load(f)
  print(manual_acts.keys())
  print(manual_acts['feature_dict'])
  # print(manual_acts['r2_score'])
  print(manual_acts['expression'])
  print(manual_acts['pred_expression'])
  break

dict_keys(['encoder', 'decoder', 'feature_dict', 'pred_expression', 'expression'])
{'exponential': 1, 'hyperbolic': 0, 'derivative_0': -0.0, 'derivative_3': -0.0, 'r2_score': 0.0}
1.0 * np.exp(-0.0 * t)
x_0' = 0.0015 * (x_0)**3



# Train probes

In [4]:
import experiments

In [8]:
target_feature = 'derivative_0'
lr = 0.01
num_epochs = 20
num_repeats = 3
layers = [idx for idx in range(0, 16)]

In [9]:
expt_results = experiments.scalar_prediction_experiment(target_feature=target_feature, activations_path=activations_path, \
                     probes_path=probes_path, \
                     lr=lr, num_epochs=num_epochs, \
                     layers=layers, num_repeats=num_repeats)
# save experiment results
with open(f'{probes_path}/experiment_data.pt', 'wb') as f:
    pickle.dump(expt_results, f)

Repeat 0 of layer 0



Training LR Probe: 100%|██████████| 20/20 [01:13<00:00,  3.68s/it]



Training Set (Epoch 20 - Final): Loss 8.80194502250821
Regression probe trained on layer 0: Test Set Loss 103.39460081563277
Saved state dictionary to /content/drive/MyDrive/aisc/probing_data/probes/derivatives/probe_derivative_0_0_0.pt
Repeat 1 of layer 0



Training LR Probe: 100%|██████████| 20/20 [01:12<00:00,  3.62s/it]



Training Set (Epoch 20 - Final): Loss 8.749565264757942
Regression probe trained on layer 0: Test Set Loss 146.07458901522207
Saved state dictionary to /content/drive/MyDrive/aisc/probing_data/probes/derivatives/probe_derivative_0_0_1.pt
Repeat 2 of layer 0


...


Training Set (Epoch 20 - Final): Loss 0.09555908142789907
Regression probe trained on layer 15: Test Set Loss 2.7995105393997886
Saved state dictionary to /content/drive/MyDrive/aisc/probing_data/probes/derivatives/probe_derivative_0_15_0.pt
Repeat 1 of layer 15



Training LR Probe: 100%|██████████| 20/20 [01:13<00:00,  3.66s/it]



Training Set (Epoch 20 - Final): Loss 0.04593772028445029
Regression probe trained on layer 15: Test Set Loss 0.5541995129075504
Saved state dictionary to /content/drive/MyDrive/aisc/probing_data/probes/derivatives/probe_derivative_0_15_1.pt
Repeat 2 of layer 15



Training LR Probe: 100%|██████████| 20/20 [01:13<00:00,  3.65s/it]



Training Set (Epoch 20 - Final): Loss 0.03836083207644668
Regression probe trained on layer 15: Test Set Loss 0.7961951217011494
Saved state dictionary to /content/drive/MyDrive/aisc/probing_data/probes/derivatives/probe_derivative_0_15_2.pt


In [10]:
expt_results

,layer,run,test_loss,final_train_loss,final_val_loss
0,0,0,103.394601,8.801945,116.119892
1,0,1,146.074589,8.749565,85.563405
2,0,2,77.113978,5.740074,67.587831
3,1,0,71.826352,4.683043,73.672558
4,1,1,61.983108,11.014737,86.045862
5,1,2,52.177637,6.044267,60.520739
6,2,0,66.445160,12.170842,162.617327
7,2,1,358.958051,22.694478,197.819234
8,2,2,587.004538,14.905535,528.887140
9,3,0,0.623346,0.002691,0.626023
